In [80]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

sale_train = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
sale_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [20]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [17]:
data.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [26]:
sale_train_fil = sale_train[sale_train['date_block_num'] ==33]
sale_train_fil.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882335,23.10.2015,33,45,13315,649.0,1.0
2882336,05.10.2015,33,45,13880,229.0,1.0
2882337,02.10.2015,33,45,13881,659.0,1.0
2882338,12.10.2015,33,45,13881,659.0,1.0
2882339,04.10.2015,33,45,13923,169.0,1.0


In [52]:
sale_group = sale_train_fil[['shop_id', 'item_id','item_cnt_day']].groupby(['shop_id','item_id']).agg({'item_cnt_day':'sum'}).reset_index()
sale_group =sale_group.rename(columns={'item_cnt_day': 'item_cnt_month'})
sale_group.head()

,shop_id,item_id,item_cnt_month
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0


In [56]:
test = pd.read_csv('test.csv')
test = pd.merge(test,sale_group, on = ['shop_id','item_id'],how ='left')
print(test.head())

   ID  shop_id  item_id  item_cnt_month
0   0        5     5037             NaN
1   1        5     5320             NaN
2   2        5     5233             1.0
3   3        5     5232             NaN
4   4        5     5268             NaN


In [57]:
test['item_cnt_month'] = test['item_cnt_month'].fillna(0).clip(0,20)
print(test.head())

   ID  shop_id  item_id  item_cnt_month
0   0        5     5037             0.0
1   1        5     5320             0.0
2   2        5     5233             1.0
3   3        5     5232             0.0
4   4        5     5268             0.0


In [58]:
test = test[['ID', 'item_cnt_month']]
submission = test.set_index('ID')

In [59]:
submission.to_csv('submission_baseline.csv')

In [84]:
shops = pd.read_csv('shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [85]:
#shops information and location

shops['shop_name'] = shops['shop_name'].apply(lambda x: x.lower()).str.replace('[^\w\s]', '').str.replace('\d+','').str.strip()
shops['shop_city'] = shops['shop_name'].str.partition(' ')[0]
shops['shop_type'] = shops['shop_name'].apply(lambda x: 'мтрц' if 'мтрц' in x else 'трц' if 'трц' in x else 'трк' if 'трк' in x else 'тц' if 'тц' in x else 'тк' if 'тк' in x else 'NO_DATA')
shops.head()

,shop_name,shop_id,shop_city,shop_type
0,якутск орджоникидзе фран,0,якутск,NO_DATA
1,якутск тц центральный фран,1,якутск,тц
2,адыгея тц мега,2,адыгея,тц
3,балашиха трк октябрькиномир,3,балашиха,трк
4,волжский тц волга молл,4,волжский,тц


In [86]:
shops['shop_city_code'] = LabelEncoder().fit_transform(shops['shop_city'])
shops['shop_type_code'] = LabelEncoder().fit_transform(shops['shop_type'])
shops.head()

,shop_name,shop_id,shop_city,shop_type,shop_city_code,shop_type_code
0,якутск орджоникидзе фран,0,якутск,NO_DATA,29,0
1,якутск тц центральный фран,1,якутск,тц,29,5
2,адыгея тц мега,2,адыгея,тц,0,5
3,балашиха трк октябрькиномир,3,балашиха,трк,1,3
4,волжский тц волга молл,4,волжский,тц,2,5


In [70]:
#sales volume for each products
sale_item_id = sale_train.pivot_table(index=['item_id'],values=['item_cnt_day'],columns='date_block_num', aggfunc=np.sum,fill_value=0).reset_index()

sale_item_id.columns = sale_item_id.columns.droplevel().map(str)
sale_item_id = sale_item_id.reset_index(drop=True).rename_axis(None, axis=1)
sale_item_id.columns.values[0]='item_id'


In [71]:
sale_item_id.tail()

,item_id,0,1,2,3,4,5,6,7,8,...,24,25,26,27,28,29,30,31,32,33
21802,22165,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21803,22166,0,0,0,0,0,0,0,0,0,...,7,8,12,4,8,10,8,11,5,11
21804,22167,0,0,0,0,0,0,0,0,56,...,33,46,40,38,31,33,34,29,21,37
21805,22168,2,2,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
21806,22169,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
#sales volume for each store

sale_shop_id = sale_train.pivot_table(index=['shop_id'],values=['item_cnt_day'],columns='date_block_num', aggfunc=np.sum,fill_value=0)

sale_shop_id.columns = sale_shop_id.columns.droplevel().map(str)
sale_shop_id = sale_shop_id.reset_index(drop=True).rename_axis(None, axis=1)

sale_shop_id.columns.values[0]='shop_id'

sale_shop_id.tail()

,shop_id,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
55,0,0,0,0,852,728,795,719,812,916,...,6429,2551,2769,3422,2117,1909,1658,1976,6867,1972
56,3238,2655,3099,2153,2140,3105,3294,3311,2829,2375,...,2444,1691,1393,1237,1315,1566,1491,1604,1194,1263
57,0,0,6586,5493,5233,5987,5827,5413,5213,5686,...,4143,3540,3113,2860,2408,2440,2352,2780,2266,2316
58,0,0,3773,2831,2308,2972,3122,2863,2735,3022,...,2519,2360,2359,1710,1378,1554,1689,1738,1319,1446
59,2017,1897,2028,1388,1374,1707,1747,2048,2008,1751,...,1293,1206,1199,1054,916,913,992,1214,914,790


In [67]:
def lag_features(df,lags,col):
    tmp =df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shift =tmp.copy()
        shift.columns =['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shift['date_block_num'] += i
        df =pd.merge(df,shift,on=['date_block_num','shop_id','item_id'], how ='left')
    return  df

In [77]:
#monthly sales volume
#ts = time.time()
matrix =[]
matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')
#time.time()- ts


TypeError: list indices must be integers or slices, not list